# Парсер для ютуб канала вдудь
[Канал](https://www.youtube.com/channel/UCMCgOm8GZkHp8zJ6l7_hIuA)

In [1]:
import pandas as pd
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException 
import time
import datetime

import googleapiclient.discovery
import requests
import pickle

Создаем таблицу для хранения данных

In [3]:
start = time.time()
cols = ['full_title', 'guest', 'length', 'views', 'likes', 'dislikes', 'date', 'comments_num']
data = []
df = pd.DataFrame(data = data, columns = cols)

In [4]:
all_Videos = 'https://www.youtube.com/c/vdud/videos'

In [5]:
driver = Chrome('C:/WebDriver/chromedriver')
driver.get(all_Videos)
driver.maximize_window()

Скроллим вниз для подгрузки всех видео

In [6]:
def scrollDown():
    driver.execute_script("window.scrollTo(0, 1660)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 3200)")
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 5000)")
    time.sleep(2)

In [7]:
scrollDown()

Собираем ссылки на все видео канала

In [8]:
play_list = driver.find_elements_by_id('thumbnail')

hrefs = [video.get_attribute('href') for video in play_list]

Некоторые ссылки дублируются

In [9]:
# It appears that some links were collected twice
hrefs = list(set(hrefs))
len(hrefs)

119

Сохраняем ссылки

In [46]:
with open('hrefs.pickle', 'wb') as f:
    pickle.dump(hrefs, f)

# Сбор данных о видео с помощью selenium

## Объвление функций

Проверяем есть ли на странице кнопка пропуска рекламы

In [10]:
def checkIfBtnExists():
    try:
        driver.find_element_by_class_name('ytp-ad-skip-button-text')
    except NoSuchElementException:
        return False
    return True

Если есть реклама пропускаем её и собираем информацию о длительности выпуска

In [11]:
def getLength():
    length = driver.find_element_by_class_name('ytp-time-duration').text
    
    if checkIfBtnExists() \
    and driver.find_element_by_class_name('ytp-ad-skip-button-text').text in ('Пропустить рекламу', 'Пропустить'):
        driver.find_element_by_class_name('ytp-ad-skip-button-text').click()
    
    if len(length)<5:
        time.sleep(1.5)
        return getLength()
    elif len(length.split(':')) == 3:
        return int(length.split(':')[0])*60 + int(length.split(':')[1])
    else:
        return int(length.split(':')[0])

Собираем название выпуска

In [12]:
def getTitle():
    full_title = driver.title
    
    title_end = full_title.find(' / ')
    if title_end < 0:
        title_end = full_title.find('.')
        if title_end < 0:
            title_end = full_title.find('(') - 1
            
    full_title = full_title[0:title_end]
    if full_title.find(' - ') > 0:
        guest = full_title[0:full_title.find('-')]
    elif full_title.find(' — ') > 0:
            guest = full_title[0:full_title.find('—')]
    elif full_title.find(' – ') > 0:
        guest = full_title[0:full_title.find('–')]
    elif full_title.find('. ') > 0:
            guest = full_title[0:full_title.find('.')]
    elif full_title.find(': ') > 0:
            guest = full_title[0:full_title.find(':')]
    else:
        guest = full_title
    return [full_title, guest]

Собираем лайки, дизлайки и просмотры

In [13]:
def getVideoStats():
    views = driver.find_element_by_class_name('yt-view-count-renderer').text
    views = int(views[0: views.find('п')-1].replace(' ', ''))
    likesDislikes = driver.find_element_by_id('top-level-buttons').text
    listLikDis = likesDislikes.replace(',', '.').split('\n')
    
    if listLikDis[0].find('ТЫС.') > 0:
        likes = float(listLikDis[0].split()[0])*1000
    elif listLikDis[0].find('МЛН') > 0:
        likes = float(listLikDis[0].split()[0])*1000000
        
    if listLikDis[1].find('ТЫС.') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000
    elif listLikDis[1].find('МЛН') > 0:
        dislikes = float(listLikDis[1].split()[0])*1000000 
        
    return [views, likes, dislikes]

Собираем дату выпуска

In [14]:
def getDate():
    date = driver.find_element_by_id('date').text.split()
    day = int(date[0][1:])
    
    months = ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек']
    for i, mon in enumerate(months):
        if mon == date[1][:3]:
            month = i+1
            
    year = int(date[2])
    return datetime.datetime(year, month, day)

Количество комментов с реплаями

In [15]:
def getCommsNum():
    driver.execute_script("window.scrollTo(0, 880)")
    time.sleep(3)
    coms = driver.find_element_by_class_name('count-text').text
    return int(coms.split()[0])*1000 + int(coms.split()[1])

Добавляем информацю о выпуске в таблицу

In [16]:
def dfAppend():
    global df
    item = [getTitle()[0], getTitle()[1], getLength(), getVideoStats()[0], getVideoStats()[1], 
          getVideoStats()[2], getDate(), getCommsNum()]
    series = pd.Series(item, index=df.columns)
    df = df.append(series, ignore_index=True)

Запускаем сбор всей информации

In [17]:
def goHrefs():
    for href in hrefs:
        if type(href) == str:
            driver.get(href)
            time.sleep(2)
            dfAppend()

## Выполнение функций

In [18]:
t = time.time()
goHrefs()
print('Time to get all the data: {} mins'.format(round((time.time() - t) / 60, 2)))

Time to get all the data: 13.63 mins


In [19]:
df.shape

(118, 8)

In [20]:
df.head()

,full_title,guest,length,views,likes,dislikes,date,comments_num
0,Максим Фадеев - о конфликте с Эрнстом и русско...,Максим Фадеев,52,8621967,167000.0,11000.0,2017-05-16,30245
1,Невзоров - о Фараоне и ориентации Милонова,Невзоров,85,14349677,326000.0,26000.0,2018-01-30,47203
2,Колокольников,Колокольников,72,3884574,79000.0,9200.0,2018-05-22,11053
3,"Собчак - о Навальном, крестном и выборах",Собчак,96,14875828,251000.0,67000.0,2017-10-24,52065
4,"Щербаков - спецназ, панк-рок, любовь",Щербаков,99,21323332,789000.0,45000.0,2020-01-28,125871


In [21]:
df['likes/dislikes'] = df['likes']/df['dislikes']
df['views/likes'] = df['views']/df['likes']
df['views/comments_num'] = df['views']/df['comments_num']

In [ ]:
df.to_csv('vdud.csv', index=False)

# Собираем комменты с помощью Youtube API

## Объявление функций

Собираем id из списка собранных ссылок

In [80]:
def getVideoIds(hrefs):
    video_ids = []
    for href in hrefs:
        if type(href) == str:
            video_ids.append(href.split('?v=')[1])
    return video_ids

API ключ

In [81]:
my_api_key = 'твой api ключ'

Собираем первые сто комментариев

In [82]:
def getComms(video_id):
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = my_api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100
    )
    init_response = request.execute()
    all_comms = []
    for item in init_response['items']:
        all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
        
    getNextComms(init_response['nextPageToken'], video_id, all_comms)
    
    return all_comms

Собираем остальные комменты

In [83]:
def getNextComms(page_token, video_id, all_comms):
    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = my_api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100,
        pageToken = page_token
    )
    
    try:
        response = request.execute()

        print(len(all_comms), end="\r")

        if len(response['items'])<99:
            for item in response['items']:
                all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
            return
        else:
            for item in response['items']:
                all_comms.append(item['snippet']['topLevelComment']['snippet']['textOriginal'])
            page_token = response['nextPageToken']
            return getNextComms(page_token, video_id, all_comms)
    except Exception as ex:
        print(ex)
        return 

Запускаем процесс сбора комментов

In [84]:
def getComsFromVideos(array_of_videos, n):
    all_video_comms = []
    for i in range(n, n+1):
        all_video_comms.append(getComms(array_of_videos[i]))
        print(i)
        print(i)
        if i==n+1:
            return all_video_comms
    return all_video_comms

## Выполнение функций

In [98]:
with open('hrefs.pickle', 'rb') as f:
    hrefs = pickle.load(f)
len(hrefs)

119

In [108]:
len(all_comms)

118

Из-за ограничений гугла на сбор информации, возможно собрать около 1 млн. комментов в сутки. Поэтому собираем комменты с 20 выпусков за день. Выводим количество собранных комментов и соотношение количества реплайев с количеством всех комментов. По соотношению реплайев и комментов проверяем много ли комментов собарли.

In [69]:
# Checking if were collected descent amount of comments through the ratio of replies to all comms
coms = list(df.comments_num[:20])
nums_no_replies = [len(text) for text in all_comms_0_20]

for i in range(len(all_comms_0_20)):
    print(df.iloc[i].guest, i)
    print(coms[i]-nums_no_replies[i])
    print('ratio of replies to all comms', (coms[i] - nums_no_replies[i])/coms[i])
    print('--------------------------------------------------')

Максим Фадеев  0
3570
ratio of replies to all comms 0.11803603901471317
--------------------------------------------------
Невзоров  1
10480
ratio of replies to all comms 0.22201978687795268
--------------------------------------------------
Колокольников 2
1858
ratio of replies to all comms 0.16809915859947525
--------------------------------------------------
Собчак  3
11095
ratio of replies to all comms 0.21309901085181984
--------------------------------------------------
Щербаков  4
7823
ratio of replies to all comms 0.062150932303707766
--------------------------------------------------
Ходорковский  5
12207
ratio of replies to all comms 0.3518374405533939
--------------------------------------------------
Влади (Каста)  6
2152
ratio of replies to all comms 0.156531859179517
--------------------------------------------------
Илья Найшуллер  7
883
ratio of replies to all comms 0.11418595629121944
--------------------------------------------------
IC3PEAK  8
9747
ratio of replies t

Сохраняем список комментов

In [105]:
with open('all_comms_0_20.pickle', 'wb') as f:
    pickle.dump(all_coms_0_20, f)

In [87]:
df = pd.read_csv('vdud.csv')

Объеденяем все комменты в один файл

In [106]:
all_comms_0_20 = pd.read_pickle('all_comms_0_20.pickle')
all_comms_21_40 = pd.read_pickle('all_comms_21_40.pickle')
all_comms_41_60 = pd.read_pickle('all_comms_41_60.pickle')
all_comms_61_80 = pd.read_pickle('all_comms_61_80.pickle')
all_comms_81_100 = pd.read_pickle('all_comms_81_100.pickle')
all_comms_101_118 = pd.read_pickle('all_comms_101_119.pickle')

all_comms = all_comms_0_20+all_comms_21_40+all_comms_41_60+all_comms_61_80+all_comms_81_100+all_comms_101_118
df['comms_text'] = all_comms

In [107]:
with open('all_comms.pickle', 'wb') as f:
    pickle.dump(all_comms, f)

Смотрим, что получилось

In [61]:
df[['guest', 'comms_text']].head(10)

,guest,comms_text
0,Максим Фадеев,[Кое-что из внутренностей:\n\n0:37 Что Фадееву...
1,Невзоров,[00:38 Что и кому Невзоров преподает в итальян...
2,Колокольников,[0:23 Сниматься в настоящем крематории в гробу...
3,Собчак,[Кое-что из внутренностей:\n\n00:24 Еще 4 меся...
4,Щербаков,[Что происходит?\n\n1:02 Щербаков стал суперзв...
5,Ходорковский,[Кое-что из внутренностей:\n\n00:29 Какое собы...
6,Влади (Каста),[Кое-что из внутренностей:\n\n1:38 Как в Росто...
7,Илья Найшуллер,[Кое-что из внутренностей:\n\n03:30 Почему кли...
8,IC3PEAK,[Еще одна ветка для на конкурса.\nНу и кое-что...
9,MORGENSHTERN,[Варианты на конкурс - сюда. Кое-что из внутре...


In [62]:
df.to_csv('vdud_coms.csv', index=False)

Дата сбора последних комментов

In [31]:
datetime.datetime.today().date()

datetime.date(2020, 11, 6)